In [1]:
# Get tinyshakespeare dataset to train the model on
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-11-25 18:18:36--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
connected. to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... 
200 OKequest sent, awaiting response... 
Length: 1115394 (1,1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1,06M  --.-KB/s    in 0,1s    

2025-11-25 18:18:36 (9,03 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# Read file
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(text[:100])

print("-----")
print("Length of dataset:{}".format(len(text)))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
-----
Length of dataset:1115394


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("".join(chars))
print("{} characters".format(vocab_size))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65 characters
